In [1]:
# Cell 1: Imports and Data Ingestion
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

# Set plotting style
plt.style.use('dark_background')

# 1. Define the Universe (The "Book")
# We mix 'Volatile' (Tech) and 'Stable' (Banks) assets
tickers = ['AAPL', 'MSFT', 'GOOGL', 'JPM', 'GS', 'MS']
benchmark = ['SPY'] # S&P 500
rates_ticker = ['^IRX'] # 3-Month Treasury Bill (Risk-Free Rate Proxy)

# 2. Fetch Data
print(f"Fetching live data...")
start_date = (datetime.now() - timedelta(days=730)).strftime('%Y-%m-%d')

# Fetch Assets & Benchmark
# auto_adjust=True automatically handles splits and dividends (No 'Adj Close' needed)
raw_data = yf.download(tickers + benchmark, start=start_date, auto_adjust=True)

# Fetch Rates
raw_rates = yf.download(rates_ticker, start=start_date, auto_adjust=True)

# 3. Data Extraction Safety Check
# yfinance sometimes returns a MultiIndex (Price Type, Ticker) or just (Ticker)
# We safely extract the 'Close' prices regardless of structure.

try:
    # Try accessing 'Close' directly if it's a MultiIndex
    data = raw_data['Close']
    risk_free_data = raw_rates['Close']
except KeyError:
    # If 'Close' isn't a top-level key, the data might already be the Close prices
    # or the structure is flat. We assume raw_data IS the dataframe we need if it's simple.
    data = raw_data
    risk_free_data = raw_rates

print("Data successfully ingested.")
display(data.tail())

Fetching live data...


[*********************100%***********************]  7 of 7 completed
[*********************100%***********************]  1 of 1 completed

Data successfully ingested.


Ticker,AAPL,GOOGL,GS,JPM,MS,MSFT,SPY
Date,,,,,,,
2026-01-29,258.279999,338.250000,940.119995,306.420013,181.399994,433.500000,694.039978
2026-01-30,259.480011,338.000000,935.409973,305.890015,182.800003,430.290009,691.969971
2026-02-02,270.010010,343.690002,946.330017,308.140015,185.100006,423.369995,695.409973
2026-02-03,269.480011,339.709991,938.989990,314.850006,182.910004,411.209991,689.530029
2026-02-04,276.489990,333.040009,913.299988,317.269989,180.080002,414.190002,686.190002


In [ ]:
# Cell 2: Data Processing & Financial Engineering

# 1. Handle Missing Data (Crucial for live feeds)
data = data.ffill().bfill() # Forward fill then backward fill to handle holidays
risk_free_data = risk_free_data.ffill().bfill()

# 2. Calculate Daily Returns (The input for Variance/Covariance Matrix)
returns = data.pct_change().dropna()

# 3. Process the Risk-Free Rate
# Convert index value (e.g., 5.2) to decimal (0.052)
risk_free_rate = risk_free_data['^IRX'] / 100
# Calculate Daily Risk-Free Rate (assuming 360-day count convention common in finance)
daily_rfr = risk_free_rate / 360

# 4. Create a "Combined Master Dataframe"
# This aligns the dates of stocks and rates exactly.
market_env = pd.concat([data, risk_free_rate], axis=1).dropna()
market_env.columns = list(data.columns) + ['Risk_Free_Rate']

print("Financial Engineering Complete.")
print(f"Current Annual Risk-Free Rate: {risk_free_rate.iloc[-1]:.2%}")
print(f"Implied Daily Funding Cost: {daily_rfr.iloc[-1]:.6%}")

# 5. Quick Visualization: Cumulative Returns
plt.figure(figsize=(14, 7))
((1 + returns[tickers]).cumprod() - 1).plot(title="Cumulative Asset Returns (2Y)")
plt.ylabel("Return Multiplier")
plt.show()

Financial Engineering Complete.
Current Annual Risk-Free Rate: 3.60%
Implied Daily Funding Cost: 0.009986%


<Figure size 1400x700 with 0 Axes>

In [1]:
# Cell 3: Create the Client Portfolio (Simulated Hedge Fund)

# 1. Define Positions (Positive = Long, Negative = Short)
# A typical "Market Neutral" style book
portfolio_dict = {
    'AAPL': 1500,   # Long 1500 shares
    'MSFT': 1000,   # Long 1000 shares
    'GOOGL': 800,   # Long 800 shares
    'JPM': -1200,   # Short 1200 shares (Betting it goes down)
    'GS': -500,     # Short 500 shares
    'MS': -1000     # Short 1000 shares
}

# 2. Convert to DataFrame
portfolio = pd.DataFrame(list(portfolio_dict.items()), columns=['Ticker', 'Shares'])

# 3. Mark-to-Market (Get latest prices)
latest_prices = data.iloc[-1] # The most recent closing prices
portfolio['Price'] = portfolio['Ticker'].map(latest_prices)

# 4. Calculate Notional Value (The dollar amount exposure)
portfolio['Notional'] = portfolio['Shares'] * portfolio['Price']

# 5. Tag Position Type
portfolio['Type'] = portfolio['Shares'].apply(lambda x: 'Long' if x > 0 else 'Short')

# Display the Book
print("Client Portfolio Snapshot:")
display(portfolio.style.format({
    'Price': '${:,.2f}',
    'Notional': '${:,.2f}'
}))

# Quick Check: Gross vs. Net Exposure
gross_exposure = portfolio['Notional'].abs().sum()
net_exposure = portfolio['Notional'].sum()
print(f"Gross Exposure (Total Risk): ${gross_exposure:,.2f}")
print(f"Net Exposure (Market Direction): ${net_exposure:,.2f}")

NameError: name 'pd' is not defined

In [2]:
# Cell 4: Calculate Daily Funding Costs & Rebates

# 1. Define Spreads (The "Price List" of the Prime Broker)
# Rates are usually in Basis Points (bps). 1 bps = 0.01%
long_spread_bps = 50      # Client pays SOFR + 50bps to borrow cash
short_rebate_spread_bps = 25 # Client receives SOFR - 25bps on short cash
stock_borrow_fee_bps = 40   # Fee to borrow the stock (Hard-to-borrow stocks cost more)

# Convert to decimals
long_spread = long_spread_bps / 10000
short_spread = short_rebate_spread_bps / 10000
borrow_fee = stock_borrow_fee_bps / 10000

# Get current Risk-Free Rate (Annualized) from our data
current_rfr = risk_free_rate.iloc[-1]

# 2. The Pricing Function
def calculate_funding(row):
    notional = row['Notional']
    
    if row['Type'] == 'Long':
        # LONG: You pay interest on the money you borrowed
        # Rate = RFR + Spread
        annual_rate = current_rfr + long_spread
        daily_cost = -1 * (notional * annual_rate) / 360 # Negative because it's a cost
        return daily_cost
        
    else: # Short
        # SHORT: Two parts
        # 1. Rebate: You earn interest on the cash proceeds (RFR - Spread)
        # 2. Borrow Fee: You pay a fee to borrow the stock
        
        # Note: Notional is negative for shorts, so we use abs() for the base calculation
        abs_notional = abs(notional)
        
        rebate_rate = current_rfr - short_spread
        rebate_earnings = (abs_notional * rebate_rate) / 360
        
        borrow_cost = (abs_notional * borrow_fee) / 360
        
        # Net Short Funding = Rebate Earnings - Borrow Cost
        return rebate_earnings - borrow_cost

# 3. Apply Logic
portfolio['Daily_Funding_Impact'] = portfolio.apply(calculate_funding, axis=1)

# 4. Annualize for easy reading
portfolio['Annual_Funding_Rate'] = portfolio['Daily_Funding_Impact'] * 360 / portfolio['Notional'].abs()

# Display with Formatting
print(f"Current Base Rate (SOFR/T-Bill): {current_rfr:.2%}")
display(portfolio.style.format({
    'Price': '${:,.2f}',
    'Notional': '${:,.2f}',
    'Daily_Funding_Impact': '${:,.2f}',
    'Annual_Funding_Rate': '{:.2%}'
}).background_gradient(subset=['Daily_Funding_Impact'], cmap='RdYlGn'))

total_daily_carry = portfolio['Daily_Funding_Impact'].sum()
print(f"Total Daily P&L from Funding (Carry): ${total_daily_carry:,.2f}")

NameError: name 'risk_free_rate' is not defined

In [ ]:
# Cell 5: Convex Optimization (The "Strategist" Engine)
import cvxpy as cp

# 1. Prepare the Data for Optimization
# We only optimize the 'Long' candidates for our Collateral Pool
long_tickers = ['AAPL', 'MSFT', 'GOOGL', 'JPM', 'GS', 'MS']
mu = returns[long_tickers].mean().values # Expected Returns (Historical)
Sigma = returns[long_tickers].cov().values # Covariance Matrix (Risk)
n_assets = len(long_tickers)

# 2. Define the Variables
# 'w' represents the weight (0% to 100%) of each stock in the optimal portfolio
w = cp.Variable(n_assets)

# 3. Define the Objective: Minimize Risk (Portfolio Variance)
# Variance = w.T * Sigma * w
risk = cp.quad_form(w, Sigma)
objective = cp.Minimize(risk)

# 4. Define Constraints
constraints = [
    cp.sum(w) == 1,  # Fully invested (Weights must sum to 100%)
    w >= 0           # Long Only (No short selling in the Collateral pool)
]

# 5. Solve the Problem
print("Running Convex Optimization...")
prob = cp.Problem(objective, constraints)
prob.solve()

# 6. Extract Results
optimal_weights = w.value
min_risk_volatility = np.sqrt(prob.value) * np.sqrt(252) # Annualized Volatility

# 7. Visualize the "Optimal" vs "Equal Weight"
opt_df = pd.DataFrame({
    'Ticker': long_tickers,
    'Optimal_Weight': np.round(optimal_weights, 4),
    'Equal_Weight': np.round([1/n_assets]*n_assets, 4)
})

print(f"Minimum Achievable Annual Volatility: {min_risk_volatility:.2%}")
print("\nOptimal Allocation (The Answer):")
display(opt_df.sort_values(by='Optimal_Weight', ascending=False))

# Chart
plt.figure(figsize=(10, 5))
sns.barplot(x='Ticker', y='Optimal_Weight', data=opt_df)
plt.title("Optimal Collateral Allocation (Minimizing Risk)")
plt.ylabel("Weight in Portfolio")
plt.show()

In [4]:
# Cell 6: Advanced "Delta One" Inventory Optimization
import cvxpy as cp

# ---------------------------------------------------------
# A. CONFIGURATION (The "Strategist" Parameters)
# ---------------------------------------------------------
GROSS_LEVERAGE_LIMIT = 2.0  # Max 2x leverage (standard PB limit)
NET_EXPOSURE_LIMIT   = 0.05 # +/- 5% Net Exposure (Almost Market Neutral)
TRANSACTION_COST_BPS = 10   # 10 basis points per trade
GAMMA_RISK_AVERSION  = 0.5  # Higher = We hate risk more than we love return

# ---------------------------------------------------------
# B. PREPARE DATA
# ---------------------------------------------------------
# We use the FULL universe now (Longs and Shorts allowed on all)
universe = tickers 
n = len(universe)

# Expected Returns & Covariance
mu = returns[universe].mean().values
Sigma = returns[universe].cov().values

# ---------------------------------------------------------
# C. THE CONVEX OPTIMIZATION MODEL (Long/Short)
# ---------------------------------------------------------
# Variable: w can now be negative (Short) or positive (Long)
w = cp.Variable(n)

# 1. Objective: Maximize Risk-Adjusted Return
# Return - (Risk_Aversion * Risk)
portfolio_return = mu @ w
portfolio_risk   = cp.quad_form(w, Sigma)
objective = cp.Maximize(portfolio_return - GAMMA_RISK_AVERSION * portfolio_risk)

# 2. Constraints (The "Prime Broker" Rules)
constraints = [
    # a. Gross Leverage Limit: Sum(|Longs| + |Shorts|) <= Limit
    # Note: cp.norm(w, 1) is the L1 norm, representing total gross exposure
    cp.norm(w, 1) <= GROSS_LEVERAGE_LIMIT,
    
    # b. Market Neutrality (Delta One): Net Weights approx 0
    # Sum(Longs) + Sum(Shorts) should be close to 0
    cp.sum(w) >= -NET_EXPOSURE_LIMIT,
    cp.sum(w) <= NET_EXPOSURE_LIMIT
]

# ---------------------------------------------------------
# D. SOLVE
# ---------------------------------------------------------
prob = cp.Problem(objective, constraints)
prob.solve()

# ---------------------------------------------------------
# E. ANALYSIS & OUTPUT
# ---------------------------------------------------------
# Clean up small numbers (floating point errors)
weights_cleaned = np.round(w.value, 4)

df_opt = pd.DataFrame({
    'Ticker': universe,
    'Weight': weights_cleaned,
    'Position': ['Long' if x > 0 else 'Short' if x < 0 else 'Flat' for x in weights_cleaned]
})

print(f" Optimization Status: {prob.status}")
print(f" Gross Leverage Used: {np.sum(np.abs(weights_cleaned)):.2f}x (Limit: {GROSS_LEVERAGE_LIMIT}x)")
print(f" Net Exposure: {np.sum(weights_cleaned):.2%}")
print("-" * 40)
print("OPTIMAL DELTA ONE INVENTORY:")
display(df_opt.sort_values(by='Weight', ascending=False).style.bar(subset=['Weight'], align='mid', color=['#d65f5f', '#5fba7d']))

NameError: name 'tickers' is not defined

In [ ]:
# Cell 7: Monte Carlo Stress Test (VaR Calculation)

# 1. Setup Simulation Parameters
days_to_simulate = 10
scenarios = 5000
initial_capital = 1_000_000  # $1 Million Portfolio

# 2. Cholesky Decomposition (To correlate random shocks)
# This ensures our random numbers move like the real market (e.g., if AAPL drops, MSFT likely drops)
L = np.linalg.cholesky(Sigma)

# 3. Run Simulation
sim_returns = []
for i in range(scenarios):
    # Generate random daily shocks for all assets
    daily_shocks = np.random.normal(size=(days_to_simulate, n))
    # Apply asset correlations
    correlated_shocks = daily_shocks @ L.T
    # Apply to our weights
    portfolio_daily_returns = correlated_shocks @ w.value
    # Compound returns over the period
    total_return = np.prod(1 + portfolio_daily_returns) - 1
    sim_returns.append(total_return)

# 4. Calculate Value at Risk (VaR)
sim_returns = np.array(sim_returns)
var_99 = np.percentile(sim_returns, 1) # Worst 1% case

# 5. Visualization
plt.figure(figsize=(12, 6))
sns.histplot(sim_returns, kde=True, color="cyan", bins=50)
plt.axvline(var_99, color='red', linestyle='--', label=f'99% VaR: {var_99:.2%}')
plt.title(f"Stress Test: 10-Day Projected P&L Distribution (N={scenarios})")
plt.xlabel("Cumulative Return")
plt.legend()
plt.show()

print(f"Risk Analysis:")
print(f"If markets crash, with 99% confidence, this portfolio will not lose more than {var_99:.2%} in 10 days.")
print(f"Monetary VaR: ${initial_capital * abs(var_99):,.2f}")

In [ ]:
# Cell 8: HIGH-FREQUENCY DATA INGESTION (1-Minute Bars)
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

# 1. The Setup: A Classic Stat-Arb Pair (Bank vs. Bank)
# We trade JPM (JPMorgan) against C (Citigroup).
# If the spread between them widens too much, we short the winner, buy the loser.
asset_x = "C"   # The Independent Variable (The Hedge)
asset_y = "JPM" # The Dependent Variable (The Target)
interval = "1m" # 1-Minute Candles (High Frequency)

print(f"⚡ Connecting to Live Markets... Fetching 1-minute data for {asset_x} vs {asset_y}")

# 2. Fetch Intraday Data (Max 7 days for 1m interval)
data_x = yf.download(asset_x, period="5d", interval=interval, auto_adjust=True)['Close']
data_y = yf.download(asset_y, period="5d", interval=interval, auto_adjust=True)['Close']

# 3. Align Microstructure
# In HFT, timestamps must match perfectly.
df_pair = pd.concat([data_x, data_y], axis=1).dropna()
df_pair.columns = ['x', 'y']

print(f"✓ Ingested {len(df_pair)} micro-bars of data.")
print("System Ready for Kalman Filtering.")

# Visual Check of the correlation
plt.figure(figsize=(12, 4))
plt.plot(df_pair['x'], label=asset_x, alpha=0.7)
plt.plot(df_pair['y'], label=asset_y, alpha=0.7)
plt.title(f"Intraday Price Action: {asset_x} vs {asset_y} (1-Min Resolution)")
plt.legend()
plt.show()

⚡ Connecting to Live Markets... Fetching 1-minute data for C vs JPM


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


✓ Ingested 1353 micro-bars of data.
System Ready for Kalman Filtering.


In [ ]:
# Cell 9: The Kalman Filter (Dynamic Beta Estimation)

class KalmanFilterReg:
    def __init__(self, delta=1e-5, R=1e-3):
        # State Covariance (P)
        self.P = np.zeros((2, 2)) 
        
        # Measurement Noise (R) - How noisy is the market?
        self.R = R 
        
        # Process Noise (Q) - How fast does the relationship change?
        # delta / (1-delta) is a standard heuristic for adaptive filters
        self.Q = np.eye(2) * delta 
        
        # State Vector (theta) -> [Intercept, Slope(Beta)]
        self.theta = np.zeros(2) 

    def update(self, x, y):
        # Observation Matrix (H) -> [1, Price_X]
        H = np.array([1, x])
        
        # 1. PREDICT STEP
        # A priori state estimate (Random Walk assumption: theta_t = theta_t-1)
        theta_pred = self.theta 
        P_pred = self.P + self.Q
        
        # 2. MEASUREMENT STEP
        # The "Innovation" (Error): Real Y - Predicted Y
        y_pred = H @ theta_pred
        error = y - y_pred
        
        # 3. UPDATE STEP (The "Kalman Gain")
        # S = System Uncertainty (H * P * H_T + R)
        S = H @ P_pred @ H.T + self.R
        
        # K = Kalman Gain (How much do we trust the new data vs old model?)
        K = P_pred @ H.T / S
        
        # Update State (theta) based on error * Gain
        self.theta = theta_pred + K * error
        
        # Update Covariance (P) - We possess more information now, so uncertainty drops
        self.P = (np.eye(2) - np.outer(K, H)) @ P_pred
        
        return self.theta, error, np.sqrt(S) # Return Beta, Spread Error, Variance

# ----------------------------------------------------
# RUN THE FILTER OVER THE DATA LOOP
# ----------------------------------------------------
kf = KalmanFilterReg(delta=1e-5, R=1e-3)

state_means = []
kalman_spreads = []
kalman_std = []

print("⚡ Running Bayesian Update Loop on Micro-Data...")

for t in range(len(df_pair)):
    x_t = df_pair['x'].iloc[t]
    y_t = df_pair['y'].iloc[t]
    
    theta, error, std = kf.update(x_t, y_t)
    
    state_means.append(theta.copy()) # Store [Alpha, Beta]
    kalman_spreads.append(error)     # The "Signal"
    kalman_std.append(std)           # The "Risk"

# Convert to DataFrame for Analysis
results = pd.DataFrame(index=df_pair.index)
results['Alpha'] = [x[0] for x in state_means]
results['Beta']  = [x[1] for x in state_means] # The Dynamic Hedge Ratio
results['Spread'] = kalman_spreads
results['Z_Score'] = results['Spread'] / kalman_std # Normalize signal

print("✓ Kalman Filter Converged.")
print(f"Current Dynamic Hedge Ratio (Beta): {results['Beta'].iloc[-1]:.4f}")

# Plot the Dynamic Beta (Show them the relationship IS NOT static)
plt.figure(figsize=(12, 6))
plt.subplot(2,1,1)
plt.plot(results['Beta'], color='#00ff00', linewidth=1)
plt.title(f"The 'Hidden' Variable: Dynamic Beta ({asset_y} vs {asset_x})")
plt.ylabel("Hedge Ratio")

plt.subplot(2,1,2)
plt.plot(results['Z_Score'], color='cyan', linewidth=0.8)
plt.axhline(2.0, color='red', linestyle='--')
plt.axhline(-2.0, color='red', linestyle='--')
plt.title("Trading Signal (Z-Score of Kalman Spread)")
plt.tight_layout()
plt.show()

In [ ]:
# Cell 10: Vectorized Backtest Engine

# 1. Define Signal Logic
results['Signal'] = 0
results.loc[results['Z_Score'] > 2.0, 'Signal'] = -1  # Short the Spread (Short Y, Long X)
results.loc[results['Z_Score'] < -2.0, 'Signal'] = 1  # Long the Spread (Long Y, Short X)
results.loc[abs(results['Z_Score']) < 0.5, 'Signal'] = 0 # Exit zone

# Forward fill positions (Hold until exit signal)
# This logic is tricky: We need to maintain position until exit condition is met.
# For simplicity in this demo, we use raw signal state (Mean Reversion strategy)
results['Position'] = results['Signal']

# 2. Calculate P&L
# Return of the Spread = Return of Y - (Beta * Return of X)
# This is the P&L of a "Hedged" portfolio
ret_y = df_pair['y'].pct_change()
ret_x = df_pair['x'].pct_change()
spread_return = ret_y - (results['Beta'].shift(1) * ret_x)

results['Strategy_Return'] = results['Position'].shift(1) * spread_return

# 3. Performance Metrics
cum_return = (1 + results['Strategy_Return'].fillna(0)).cumprod()
total_return = cum_return.iloc[-1] - 1
sharpe_ratio = results['Strategy_Return'].mean() / results['Strategy_Return'].std() * np.sqrt(252 * 390) # Annualized for 1m bars

print("="*40)
print(f"ALGORITHMIC TRADING RESULTS ({interval} Bars)")
print("="*40)
print(f"Total Return (5 Days): {total_return:.4%}")
print(f"Sharpe Ratio (Annualized): {sharpe_ratio:.2f}")

# 4. The Money Chart
plt.figure(figsize=(12, 5))
cum_return.plot(color='gold')
plt.title(f"Cumulative P&L: Kalman Mean Reversion Strategy")
plt.ylabel("Growth of $1")
plt.grid(alpha=0.2)
plt.show()

In [ ]:
%%writefile app.py
import streamlit as st
import yfinance as yf
import pandas as pd
import numpy as np
import cvxpy as cp
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

# -----------------------------------------------------------------------------
# 1. PAGE CONFIGURATION (The "Temple Gate")
# -----------------------------------------------------------------------------
st.set_page_config(
    page_title="ASTRA | BNPP GMQR System",
    page_icon="🦁",
    layout="wide",
    initial_sidebar_state="expanded"
)

# Custom CSS for "Dark Mode" Finance Aesthetics
st.markdown("""
    <style>
    .metric-card {background-color: #1e1e1e; border-radius: 10px; padding: 20px; color: white;}
    .stAlert {background-color: #2b2b2b; color: #e0e0e0;}
    </style>
    """, unsafe_allow_html=True)

st.title("🦁 ASTRA: Adaptive Statistical Trading & Risk Architecture")
st.markdown("### **Global Markets Quantitative Research (GMQR) | Prime Strategist**")

# -----------------------------------------------------------------------------
# 2. SHARED DATA INGESTION ENGINE
# -----------------------------------------------------------------------------
@st.cache_data
def get_data(tickers, period="2y", interval="1d"):
    """Fetches data and automatically handles splits/dividends."""
    try:
        data = yf.download(tickers, period=period, interval=interval, auto_adjust=True)['Close']
        return data
    except Exception as e:
        st.error(f"Data Connection Failed: {e}")
        return pd.DataFrame()

# Sidebar Controls
st.sidebar.header("System Controls")
mode = st.sidebar.radio("Select Module", ["🛡️ Prime Brokerage (Inventory)", "⚡ Alpha Engine (Kalman Filter)"])

# -----------------------------------------------------------------------------
# MODULE A: PRIME BROKERAGE INVENTORY OPTIMIZER
# -----------------------------------------------------------------------------
if mode == "🛡️ Prime Brokerage (Inventory)":
    st.subheader("🛡️ Delta One Inventory & Funding Optimization")
    
    col1, col2, col3 = st.columns(3)
    with col1:
        st.info("**Objective:** Minimize Portfolio Volatility under strict Balance Sheet constraints.")
    with col2:
        leverage_limit = st.slider("Gross Leverage Limit (x)", 1.0, 5.0, 2.0, 0.1)
    with col3:
        gamma_risk = st.slider("Risk Aversion Parameter (Gamma)", 0.1, 10.0, 1.0)

    # 1. Define Universe (Hardcoded 'The Book' for demo)
    prime_tickers = ['AAPL', 'MSFT', 'GOOGL', 'JPM', 'GS', 'MS', 'TSLA', 'NVDA']
    with st.spinner(f"Connecting to Live Markets... Fetching {prime_tickers}"):
        df_prime = get_data(prime_tickers)
        
    if not df_prime.empty:
        # Calculate Returns
        returns = df_prime.pct_change().dropna()
        mu = returns.mean().values
        Sigma = returns.cov().values
        n = len(prime_tickers)

        # 2. CONVEX OPTIMIZATION (CVXPY)
        w = cp.Variable(n)
        portfolio_return = mu @ w
        portfolio_risk = cp.quad_form(w, Sigma)
        
        # Maximize Utility: Return - (Gamma * Risk)
        objective = cp.Maximize(portfolio_return - gamma_risk * portfolio_risk)
        
        # Constraints: 
        # 1. Gross Leverage <= Limit (The Bank's Balance Sheet)
        # 2. Net Exposure approx 0 (Delta Neutral)
        constraints = [
            cp.norm(w, 1) <= leverage_limit,
            cp.sum(w) >= -0.10, # Allow +/- 10% drift
            cp.sum(w) <= 0.10
        ]
        
        prob = cp.Problem(objective, constraints)
        prob.solve()
        
        # 3. OUTPUT
        weights = np.round(w.value, 4)
        
        # Metrics Row
        m1, m2, m3 = st.columns(3)
        m1.metric("Gross Leverage Used", f"{np.sum(np.abs(weights)):.2f}x", f"Limit: {leverage_limit}x")
        m2.metric("Net Exposure (Delta)", f"{np.sum(weights):.2%}", "Target: 0%")
        m3.metric("projected Annual Volatility", f"{np.sqrt(prob.value)*np.sqrt(252):.2%}")
        
        # Visualization
        results_df = pd.DataFrame({'Ticker': prime_tickers, 'Weight': weights})
        results_df['Type'] = results_df['Weight'].apply(lambda x: 'Long' if x > 0 else 'Short')
        
        fig, ax = plt.subplots(figsize=(10, 5), facecolor='#0e1117')
        sns.barplot(data=results_df, x='Ticker', y='Weight', hue='Type', palette={'Long': '#00ff00', 'Short': '#ff0000'}, dodge=False, ax=ax)
        ax.set_facecolor('#0e1117')
        ax.axhline(0, color='white', linewidth=1)
        ax.tick_params(colors='white')
        ax.set_title("Optimal Inventory Allocation (Convex Solution)", color='white')
        st.pyplot(fig)
        
        st.dataframe(results_df.style.background_gradient(cmap='RdYlGn', subset=['Weight']))

# -----------------------------------------------------------------------------
# MODULE B: KALMAN FILTER ALPHA ENGINE
# -----------------------------------------------------------------------------
elif mode == "⚡ Alpha Engine (Kalman Filter)":
    st.subheader("⚡ High-Frequency Statistical Arbitrage (Kalman Filter)")
    st.markdown("Dynamic Beta estimation using Bayesian Inference on Intraday Data.")

    c1, c2 = st.columns(2)
    with c1:
        asset_x = st.text_input("Asset X (Independent/Hedge)", "C")
    with c2:
        asset_y = st.text_input("Asset Y (Target/Dependent)", "JPM")
        
    # FETCH INTRADAY DATA
    if st.button("Initialize Kalman Algorithm"):
        with st.spinner("Ingesting Micro-structure Data (1-Minute Intervals)..."):
            # Fetch 5 days of 1-minute data
            dx = yf.download(asset_x, period="5d", interval="1m", auto_adjust=True)['Close']
            dy = yf.download(asset_y, period="5d", interval="1m", auto_adjust=True)['Close']
            
            # Align
            df_pair = pd.concat([dx, dy], axis=1).dropna()
            df_pair.columns = ['x', 'y']
            
            # -----------------------------------------------------------
            # THE KALMAN FILTER CLASS (Embedded for Production)
            # -----------------------------------------------------------
            class KalmanFilterReg:
                def __init__(self, delta=1e-5, R=1e-3):
                    self.P = np.zeros((2, 2)) 
                    self.R = R 
                    self.Q = np.eye(2) * delta 
                    self.theta = np.zeros(2) 

                def update(self, x, y):
                    H = np.array([1, x])
                    theta_pred = self.theta 
                    P_pred = self.P + self.Q
                    y_pred = H @ theta_pred
                    error = y - y_pred
                    S = H @ P_pred @ H.T + self.R
                    K = P_pred @ H.T / S
                    self.theta = theta_pred + K * error
                    self.P = (np.eye(2) - np.outer(K, H)) @ P_pred
                    return self.theta, error, np.sqrt(S)

            # Run Filter
            kf = KalmanFilterReg()
            betas = []
            z_scores = []
            
            progress_bar = st.progress(0)
            
            for t in range(len(df_pair)):
                theta, error, std = kf.update(df_pair['x'].iloc[t], df_pair['y'].iloc[t])
                betas.append(theta[1])
                z_scores.append(error / std)
                if t % 100 == 0:
                    progress_bar.progress(t / len(df_pair))
                    
            progress_bar.empty()
            
            # Plotting
            st.success(f"Algorithm Converged. Current Dynamic Beta: {betas[-1]:.4f}")
            
            # Dynamic Beta Chart
            fig2, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8), sharex=True, facecolor='#0e1117')
            
            ax1.plot(betas, color='#00ff00', linewidth=1)
            ax1.set_title(f"Real-Time Hedge Ratio (Beta)", color='white')
            ax1.set_facecolor('#0e1117')
            ax1.tick_params(colors='white')
            
            ax2.plot(z_scores, color='cyan', linewidth=0.8)
            ax2.axhline(2.0, color='red', linestyle='--')
            ax2.axhline(-2.0, color='red', linestyle='--')
            ax2.set_title("Z-Score Trading Signal (Mean Reversion)", color='white')
            ax2.set_facecolor('#0e1117')
            ax2.tick_params(colors='white')
            
            st.pyplot(fig2)

st.sidebar.markdown("---")
st.sidebar.info("Developed by: The 10th House Architect\nTarget: BNP Paribas GMQR")

In [ ]:
# 1. Export the Optimized Inventory (The "Answer")
opt_df.to_csv('Optimized_Inventory_Weights.csv', index=False) # 

# 2. Export the Kalman Strategy Signals (The "Alpha")
results.to_csv('Kalman_Trading_Signals.csv') # 

# 3. Export the Monte Carlo Results (The "Risk")
pd.DataFrame(sim_returns, columns=['Simulated_Returns']).to_csv('Stress_Test_Scenarios.csv') # 

print("Master data files have been manifested in your directory.")